# EXERCISE 15

Download heart disease dataset heart.csv in Exercise folder and do following, (credits of dataset: https://www.kaggle.com/fedesoriano/heart-failure-prediction)

1. Load heart disease dataset in pandas dataframe
2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
3. Convert text columns to numbers using label encoding and one hot encoding
4. Apply scaling
5. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
6. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
7. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('heart_disease.csv')
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
data.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [4]:
data.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


# Data Cleaning 

### Handling Outliers

In [5]:
data.RestingBP.mean() + 3 * data.RestingBP.std()

187.93897652094344

In [6]:
data[data.RestingBP>(data.RestingBP.mean() * 3 + data.RestingBP.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [7]:
data[data.Cholesterol>(data.Cholesterol.mean() * 3 + data.Cholesterol.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [8]:
data[data.MaxHR>(data.MaxHR.mean() * 3 + data.MaxHR.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease


In [9]:
data[data.Oldpeak>(data.Oldpeak.mean() * 3 + data.Oldpeak.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
68,52,M,ASY,160,246,0,ST,82,Y,4.0,Flat,1
166,50,M,ASY,140,231,0,ST,140,Y,5.0,Flat,1
500,65,M,ASY,136,248,0,Normal,140,Y,4.0,Down,1
521,61,M,ASY,120,282,0,ST,135,Y,4.0,Down,1
537,74,M,ASY,150,258,1,ST,130,Y,4.0,Down,1
559,64,M,ASY,134,273,0,Normal,102,Y,4.0,Down,1
624,63,F,ASY,150,407,0,LVH,154,N,4.0,Flat,1
702,59,M,TA,178,270,0,LVH,145,N,4.2,Down,0
732,56,F,ASY,200,288,1,LVH,133,Y,4.0,Down,1
771,55,M,ASY,140,217,0,Normal,111,Y,5.6,Down,1


In [10]:
newdata = data[data.Oldpeak<=(data.Oldpeak.mean() * 3 + data.Oldpeak.std())]

In [11]:
data.shape

(918, 12)

In [12]:
newdata.shape

(903, 12)

### Handling Text data

In [13]:
newdata.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


1. Sex, ChestPainType are **Nominal Variables**
2. RestingECG, ExerciseAngina, ST_Slope are **Ordinal Variables**

#### Label Encoding on Ordinal Variables

In [14]:
label_encoded_data = newdata.copy()

In [15]:
label_encoded_data.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [16]:
label_encoded_data.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [17]:
label_encoded_data.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [18]:
label_encoded_data.RestingECG.replace(
{
    "Normal" : 1,
    "ST" : 2,
    "LVH" : 3
}, inplace=True)

label_encoded_data.ExerciseAngina.replace({
    "N" : 0,
    "Y" : 1
}, inplace=True)

label_encoded_data.ST_Slope.replace({
    "Up" : 1,
    "Flat" : 2,
    "Down" : 3
}, inplace=True)

In [19]:
label_encoded_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.0,1,0
1,49,F,NAP,160,180,0,1,156,0,1.0,2,1
2,37,M,ATA,130,283,0,2,98,0,0.0,1,0
3,48,F,ASY,138,214,0,1,108,1,1.5,2,1
4,54,M,NAP,150,195,0,1,122,0,0.0,1,0


#### Dummy Variable on Nominal Variables

In [20]:
finaldata = pd.get_dummies(label_encoded_data, drop_first=True).astype(int)
finaldata.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0,1,0,1,1,0,0
1,49,160,180,0,1,156,0,1,2,1,0,0,1,0
2,37,130,283,0,2,98,0,0,1,0,1,1,0,0
3,48,138,214,0,1,108,1,1,2,1,0,0,0,0
4,54,150,195,0,1,122,0,0,1,0,1,0,1,0


# X and y Split

In [21]:
X = finaldata.drop('HeartDisease', axis=1)
y = finaldata.HeartDisease

# Scaling

In [22]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.42477763,  0.42581651,  0.83172252, ...,  2.05417944,
        -0.53851648, -0.22632356],
       [-0.47034573,  1.51477802, -0.16192881, ..., -0.48681239,
         1.85695338, -0.22632356],
       [-1.7429216 , -0.11866424,  0.77702611, ...,  2.05417944,
        -0.53851648, -0.22632356],
       ...,
       [ 0.37803818, -0.11866424, -0.60861611, ..., -0.48681239,
        -0.53851648, -0.22632356],
       [ 0.37803818, -0.11866424,  0.34857095, ...,  2.05417944,
        -0.53851648, -0.22632356],
       [-1.63687361,  0.31692036, -0.20750915, ..., -0.48681239,
         1.85695338, -0.22632356]])

# Train Test split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y,
                                                   random_state=30)

In [24]:
len(X_train)

722

In [25]:
len(X_test)

181

# Model

### SVM Standalone

In [26]:
model = SVC()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8839779005524862

In [27]:
np.mean(cross_val_score(SVC(), X_scaled, y, cv=5))

0.811669736034377

### Using GridSearchCV to find best params for SVM

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
clf = GridSearchCV(SVC(gamma='auto'), {
    'C' : [1, 3, 5, 7, 10, 20],
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']
}, cv=5, return_train_score=False)

In [30]:
clf.fit(X_scaled, y)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 3, 5, 7, 10, 20],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [34]:
clf_res = pd.DataFrame(clf.cv_results_)
clf_res
clf_res[['param_C','param_kernel', 'mean_test_score', 'rank_test_score']]

,param_C,param_kernel,mean_test_score,rank_test_score
0,1,linear,0.812781,5
1,1,poly,0.801774,10
2,1,rbf,0.811670,8
3,1,sigmoid,0.818330,1
4,3,linear,0.813886,2
5,3,poly,0.799546,11
6,3,rbf,0.809472,9
7,3,sigmoid,0.777336,20
8,5,linear,0.813886,2
9,5,poly,0.793984,14


In [36]:
clf.best_params_

{'C': 1, 'kernel': 'sigmoid'}

In [41]:
clf.best_score_

0.8183302639656229

In [68]:
np.mean(cross_val_score(SVC(C=1, kernel='sigmoid', gamma='auto'), X_scaled, y, cv=5))

0.8183302639656229

# Bagging

# SVM

In [69]:
bag_svm = BaggingClassifier(
    base_estimator=SVC(C=1, kernel='sigmoid', gamma='auto'),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)

In [70]:
bag_svm.fit(X_train, y_train)

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=SVC(C=1, gamma='auto', kernel='sigmoid'),
                  max_samples=0.8, n_estimators=100, oob_score=True,
                  random_state=0)

In [71]:
bag_svm.oob_score_

0.8421052631578947

In [72]:
bag_svm.score(X_test, y_test)

0.8729281767955801

# Decision Tree

In [73]:
np.mean(cross_val_score(DecisionTreeClassifier(), X_scaled, y, cv=5))

0.7230325352977286

In [74]:
bag_decisiontree = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)


In [76]:
bag_decisiontree.fit(X_train, y_train)

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=100, oob_score=True, random_state=0)

In [78]:
bag_decisiontree.oob_score_

0.8462603878116344

In [79]:
bag_decisiontree.score(X_test, y_test)

0.8674033149171271